In [28]:
##paso 1##
import pandas as pd
import numpy as np

dataset_test = pd.read_csv(
    "../files/input/test_data.csv.zip",
    index_col=False,
    compression="zip",
)
dataset_train = pd.read_csv(
    "../files/input/train_data.csv.zip",
    index_col=False,
    compression="zip",
)

# - Renombre la columna "default payment next month" a "default".
dataset_test.rename(columns={"default payment next month": "default"}, inplace=True)
dataset_train.rename(columns={"default payment next month": "default"}, inplace=True)

# - Remueva la columna "ID".
dataset_test.drop(columns=["ID"], inplace=True)
dataset_train.drop(columns=["ID"], inplace=True)

# - Elimine los registros con informacion no disponible.
dataset_test.dropna(inplace=True)
dataset_train.dropna(inplace=True)

# - Para la columna EDUCATION, valores > 4 indican niveles superiores
#   de educación, agrupe estos valores en la categoría "others".
dataset_test.loc[dataset_test["EDUCATION"] > 4, "EDUCATION"] = 4
dataset_train.loc[dataset_train["EDUCATION"] > 4, "EDUCATION"] = 4

dataset_train


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,310000,1,3,1,32,0,0,0,0,0,...,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
1,10000,2,3,1,49,-1,-1,-2,-1,2,...,1690,1138,930,0,0,2828,0,182,0,1
2,50000,1,2,1,28,-1,-1,-1,0,-1,...,45975,1300,43987,0,46257,2200,1300,43987,1386,0
3,80000,2,3,1,52,2,2,3,3,3,...,40748,39816,40607,3700,1600,1600,0,1600,1600,1
4,270000,1,1,2,34,1,2,0,0,2,...,22448,15490,17343,0,4000,2000,0,2000,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,140000,2,2,1,27,2,-1,-1,-1,0,...,1580,804,728,752,800,1580,0,700,700,0
20996,130000,1,2,2,41,0,0,0,0,0,...,123107,42897,39378,4442,5200,5012,2500,5000,2000,0
20997,50000,1,3,2,23,0,0,0,0,0,...,28967,29829,30046,1973,1426,1001,1432,1062,997,0
20998,90000,2,3,2,25,0,0,0,0,0,...,5613,10113,10113,3000,3000,0,4500,0,3440,0


In [30]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.
y_train=dataset_train.copy()
y_train=y_train.pop("default")

x_train = dataset_train.copy()
x_train.pop("default")

y_test=dataset_test.copy()
y_test=y_test.pop("default")

x_test = dataset_test.copy()
x_test.pop("default")

0       1
1       0
2       0
3       0
4       0
       ..
8995    0
8996    0
8997    0
8998    1
8999    1
Name: default, Length: 9000, dtype: int64

In [33]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import pandas as pd


#'categorical_features' es una lista de nombres de columnas categóricas
categorical_features = ["SEX", "EDUCATION", "MARRIAGE"] 

# Crear el preprocesador que aplica el transformador a las columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
    ], remainder='passthrough'
    )

# Crear el pipeline que incluye el preprocesador y el modelo de Random Forest
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])


# - Ajusta un modelo de bosques aleatorios (rando forest).
pipeline.fit(x_train, y_train)
predictions = pipeline.predict(x_test)

predictions


array([0, 0, 0, ..., 0, 1, 0], shape=(9000,))

In [34]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.
#
# Guarde el mejor modelo en la variable "best_model".
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score

# Definir los hiperparametros a optimizar
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [10, 20, 30],
}

# Definir la métrica de evaluación
scorer = make_scorer(balanced_accuracy_score)

# Realizar la búsqueda de hiperparametros
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring=scorer,
    cv=10,
    n_jobs=-1
)

# Ajustar el modelo
grid_search.fit(x_train, y_train)

# Guardar el mejor modelo
best_model = grid_search.best_estimator_



KeyboardInterrupt: 